In [27]:
import os
import sys
import statistics

aoc_year, aoc_day = os.getcwd().split(os.sep)[-2:]

# Download today puzzle & input
!aoc --version
!aoc download -i input.txt --overwrite -p README.md --year {aoc_year} --day {aoc_day}

aoc-cli 0.6.0
Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 13, 2022...
Saving puzzle description to "README.md"...
Saving puzzle input to "input.txt"...
Done!


\--- Day 13: Distress Signal ---
----------

You climb the hill and again try contacting the Elves. However, you instead receive a signal you weren't expecting: a *distress signal*.

Your handheld device must still not be working properly; the packets from the distress signal got decoded *out of order*. You'll need to re-order the list of received packets (your puzzle input) to decode the message.

Your list consists of pairs of packets; pairs are separated by a blank line. You need to identify *how many pairs of packets are in the right order*.

For example:

```
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]

```

Packet data consists of lists and integers. Each list starts with `[`, ends with `]`, and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called *left* and the second value is called *right*. Then:

* If *both values are integers*, the *lower integer* should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.
* If *both values are lists*, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
* If *exactly one value is an integer*, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing `[0,0,0]` and `2`, convert the right value to `[2]` (a list containing `2`); the result is then found by instead comparing `[0,0,0]` and `[2]`.

Using these rules, you can determine which of the pairs in the example are in the right order:

```
== Pair 1 ==
- Compare [1,1,3,1,1] vs [1,1,5,1,1]
  - Compare 1 vs 1
  - Compare 1 vs 1
  - Compare 3 vs 5
    - Left side is smaller, so inputs are in the right order

== Pair 2 ==
- Compare [[1],[2,3,4]] vs [[1],4]
  - Compare [1] vs [1]
    - Compare 1 vs 1
  - Compare [2,3,4] vs 4
    - Mixed types; convert right to [4] and retry comparison
    - Compare [2,3,4] vs [4]
      - Compare 2 vs 4
        - Left side is smaller, so inputs are in the right order

== Pair 3 ==
- Compare [9] vs [[8,7,6]]
  - Compare 9 vs [8,7,6]
    - Mixed types; convert left to [9] and retry comparison
    - Compare [9] vs [8,7,6]
      - Compare 9 vs 8
        - Right side is smaller, so inputs are not in the right order

== Pair 4 ==
- Compare [[4,4],4,4] vs [[4,4],4,4,4]
  - Compare [4,4] vs [4,4]
    - Compare 4 vs 4
    - Compare 4 vs 4
  - Compare 4 vs 4
  - Compare 4 vs 4
  - Left side ran out of items, so inputs are in the right order

== Pair 5 ==
- Compare [7,7,7,7] vs [7,7,7]
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Right side ran out of items, so inputs are not in the right order

== Pair 6 ==
- Compare [] vs [3]
  - Left side ran out of items, so inputs are in the right order

== Pair 7 ==
- Compare [[[]]] vs [[]]
  - Compare [[]] vs []
    - Right side ran out of items, so inputs are not in the right order

== Pair 8 ==
- Compare [1,[2,[3,[4,[5,6,7]]]],8,9] vs [1,[2,[3,[4,[5,6,0]]]],8,9]
  - Compare 1 vs 1
  - Compare [2,[3,[4,[5,6,7]]]] vs [2,[3,[4,[5,6,0]]]]
    - Compare 2 vs 2
    - Compare [3,[4,[5,6,7]]] vs [3,[4,[5,6,0]]]
      - Compare 3 vs 3
      - Compare [4,[5,6,7]] vs [4,[5,6,0]]
        - Compare 4 vs 4
        - Compare [5,6,7] vs [5,6,0]
          - Compare 5 vs 5
          - Compare 6 vs 6
          - Compare 7 vs 0
            - Right side is smaller, so inputs are not in the right order

```

What are the indices of the pairs that are already *in the right order*? (The first pair has index 1, the second pair has index 2, and so on.) In the above example, the pairs in the right order are 1, 2, 4, and 6; the sum of these indices is `*13*`.

Determine which pairs of packets are already in the right order. *What is the sum of the indices of those pairs?*


In [28]:
from pprint import pprint

with open('input.txt', 'rt') as f:
    lines = [x.strip() for x in f.readlines()]

# Verify parse
pprint(lines[:6])

['[[1,0,[0]]]',
 '[[[8,[10]],7],[[4,[2,0,10,9,6],7,[2]],[[7,2,1,7,0],[9,4,5]],[6,6,[5,1,2,1,7],[8,0,6]]],[[],3,[[4,1,7,6,0]],6],[[[1,0],9,[10,7,9,8,8]],[[1,1,5,2]]]]',
 '',
 '[[[[0,6,2],[8,7,7],5,9],3],[[],[[6,1,10,6,2],1,0]]]',
 '[[[1,[0],[3,10,0,8],[1,7]],0,0,[[5,8,3],[4,7],10]]]',
 '']


In [29]:
def parse_packets(lines):
    packets = []
    while len(lines) > 0:
        packet = (
            eval(lines.pop(0)),
            eval(lines.pop(0))
        )
        packets.append(packet)
        if len(lines) > 0: lines.pop(0)
    return packets

packets = parse_packets(lines.copy())

In [30]:
def packet_compare(left, right, depth_stop = 100):
    assert depth_stop > 0, f"Max recursion: left={left} right={right}"
    # Enure lists
    left = left if isinstance(left, list) else [left]
    right = right if isinstance(right, list) else [right]
    
    # Iterate through lists
    for ileft, iright in zip(left, right):
        if isinstance(ileft, list) or isinstance(iright, list):
            # Compare as list elements
            diff = packet_compare(ileft, iright, depth_stop-1)
        else:
            # Compare integer values
            diff = iright - ileft
        if diff != 0:
            return diff
    # Fallback to length of lists - right should be longer
    return len(right)-len(left)

# Validation
print((res:=packet_compare([1,1,3,1,1], [1,1,5,1,1])) > 0, res)
print((res:=packet_compare([[1],[2,3,4]], [[1],4])) > 0, res)
print((res:=packet_compare([9], [[8,7,6]])) < 0, res)
print((res:=packet_compare([[4,4],4,4], [[4,4],4,4,4])) > 0, res)
print((res:=packet_compare([7,7,7,7], [7,7,7])) < 0, res)
print((res:=packet_compare([], [3])) > 0, res)
print((res:=packet_compare([[[]]], [[]])) < 0, res)
print((res:=packet_compare([1,[2,[3,[4,[5,6,7]]]],8,9], [1,[2,[3,[4,[5,6,0]]]],8,9])) < 0, res)

True 2
True 2
True -1
True 1
True -1
True 1
True -1
True -7


In [31]:
correct = []
for i, (left, right) in enumerate(packets):
    valid = packet_compare(left, right)
    #print(f"{i}: {valid}")
    if valid>0: correct.append(i+1)

In [32]:
answer1 = sum(correct)
print("answer 1:", answer1)

answer 1: 5506


----

In [33]:
# Download part 2
!aoc download --description-only --overwrite --puzzle-file README.md --year {aoc_year} --day {aoc_day}

Loaded session cookie from "/home/kev/.adventofcode.session".
Fetching puzzle for day 13, 2022...
Saving puzzle description to "README.md"...
Done!


\--- Part Two ---
----------

Now, you just need to put *all* of the packets in the right order. Disregard the blank lines in your list of received packets.

The distress signal protocol also requires that you include two additional *divider packets*:

```
[[2]]
[[6]]

```

Using the same rules as before, organize all packets - the ones in your list of received packets as well as the two divider packets - into the correct order.

For the example above, the result of putting the packets in the correct order is:

```
[]
[[]]
[[[]]]
[1,1,3,1,1]
[1,1,5,1,1]
[[1],[2,3,4]]
[1,[2,[3,[4,[5,6,0]]]],8,9]
[1,[2,[3,[4,[5,6,7]]]],8,9]
[[1],4]
[[2]]
[3]
[[4,4],4,4]
[[4,4],4,4,4]
[[6]]
[7,7,7]
[7,7,7,7]
[[8,7,6]]
[9]

```

Afterward, locate the divider packets. To find the *decoder key* for this distress signal, you need to determine the indices of the two divider packets and multiply them together. (The first packet is at index 1, the second packet is at index 2, and so on.) In this example, the divider packets are *10th* and *14th*, and so the decoder key is `*140*`.

Organize all of the packets into the correct order. *What is the decoder key for the distress signal?*


In [47]:
import functools

packets = parse_packets(lines.copy())
# Unpack packets
distress_signal = [p[0] for p in packets] + [p[1] for p in packets]
# Insert divider packets
distress_signal.append([[2]])
distress_signal.append([[6]])

sorted_packets = sorted(distress_signal, key=functools.cmp_to_key(packet_compare), reverse=True)

# Find indices (1-based)
i2 = sorted_packets.index([[2]]) + 1
i6 = sorted_packets.index([[6]]) + 1

print(i2, i6)

111 196


In [48]:
answer2 = i2 * i6
print("answer 2:", answer2)

answer 2: 21756
